In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#df = pd.read_csv(‘xx.csv’)

In [3]:
path = os.path.join(os.getcwd(), '../raw_data')

data_prep = pd.read_csv(os.path.join(path, 'x_train_preprocessed.csv'))
data_prep

,Unnamed: 0,summary,positives,negatives,advice_to_mgmt,review,summary_length,postives_length,negatives_length,advice_length,...,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,0,good job considering retail,apple good company period pay average working ...,job movement lateral extremely difficult get p...,NaN,good job considering retail apple good company...,31,118,127,0,...,0.600000,0.700000,0.666667,0.516667,1.000000,-0.500000,0.000000,0.000000,0.720000,0.350000
1,1,warning potential new hire,good pay smart coworkers good insurance,hr protect company employee careful restricted...,many employment contract state restricted stoc...,warning potential new hire good pay smart cowo...,31,41,132,1509,...,0.727273,0.068182,0.614286,0.538095,0.625000,0.075000,0.499852,0.120897,0.536005,0.157255
2,2,demanding job get work brilliant mind industry,intelligent co worker encouraged innovate good...,promote work life balance word employee get re...,employee get burned soon life outside work goi...,demanding job get work brilliant mind industry...,87,89,164,150,...,1.000000,0.900000,0.833333,0.783333,0.000000,0.000000,0.368750,-0.156250,0.706250,0.489583
3,3,deliver get fired,pay competitive look good resume move around i...,work life balance hostile environment reward b...,people disposable,deliver get fired pay competitive look good re...,21,189,306,26,...,0.000000,0.000000,0.460714,0.403571,0.492063,0.031746,0.000000,0.000000,0.474150,0.244218
4,4,good salary perk benefit,good salary perk benefit,hectic schedule u ill learning cutting technology,NaN,good salary perk benefit good salary perk bene...,34,34,57,0,...,0.600000,0.700000,0.600000,0.700000,0.950000,-0.550000,0.000000,0.000000,0.775000,0.075000
5,5,happy year working amazon,great people challenging project fast paced en...,hard get often turn away good people left clea...,great hiring need get better,happy year working amazon great people challen...,45,59,108,50,...,1.000000,0.800000,0.783333,0.500000,0.381250,0.127083,0.625000,0.650000,0.612500,0.410833
6,6,one hopefully soon remembered,great experience advancement understanding uti...,always enough time meet deadline,keep positive attitude try cooperate much poss...,one hopefully soon remembered great experience...,35,121,40,80,...,0.000000,0.000000,0.575000,0.450000,0.500000,0.000000,0.772727,0.113636,0.639091,0.225455
7,7,hardwork peanut,fellow team member great work,limited opportunity move within company,try treat team member like robot like human,hardwork peanut fellow team member great work ...,20,44,50,67,...,0.000000,0.000000,0.750000,0.800000,0.142857,-0.071429,0.100000,0.000000,0.330952,0.242857
8,8,software developer,really nothing share v,everything terrible v,NaN,software developer really nothing share v ever...,18,26,28,0,...,0.000000,0.000000,0.200000,0.200000,1.000000,-1.000000,0.000000,0.000000,0.600000,-0.400000
9,9,everyone high stress company growing pain,world class benefit smart coworkers food galor...,work life balance unrealistic recruiting expec...,stop micromanaging sourcers current recruiting...,everyone high stress company growing pain worl...,56,60,235,238,...,0.540000,0.160000,0.421429,0.257143,0.644444,-0.300000,0.594444,-0.133333,0.566614,-0.069524


In [4]:
negatives = data_prep.negatives.dropna()
negatives

0        job movement lateral extremely difficult get p...
1        hr protect company employee careful restricted...
2        promote work life balance word employee get re...
3        work life balance hostile environment reward b...
4        hectic schedule u ill learning cutting technology
5        hard get often turn away good people left clea...
6                         always enough time meet deadline
7                  limited opportunity move within company
8                                    everything terrible v
9        work life balance unrealistic recruiting expec...
10          growing inertia mono culture better learn love
11                          con find big corporate company
12          getting bureaucratic many reporting goal clear
13                                           find con till
14       getting promoted depends know loud one would n...
15       high pressure bad treatment really bad hour re...
16       sure start bad management bad attitude recogni.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [6]:
#vectorizer = TfidfVectorizer(max_df=0.8, ngram_range=(2,2)).fit(negatives) # words that appear most (top 20%) left out

#data_vectorized = vectorizer.transform(negatives)

#lda_model = LatentDirichletAllocation(n_components=3).fit(data_vectorized)

#print(lda_model, vectorizer)

In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(negatives_tokenized, min_count=5, threshold=100) # higher threshold fewer phrases.
#trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
#trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(bigram_mod[bigram_mod[negatives_tokenized]])

NameError: name 'gensim' is not defined

In [ ]:
bigram_mod

In [ ]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [ ]:
make_bigrams(negatives_tokenized)

In [ ]:
from nltk import word_tokenize
def tokenize(df):
    tokenized_text = word_tokenize(str(df))
    return tokenized_text

In [ ]:
negatives_tokenized = negatives.apply(tokenize)
negatives_tokenized[0]

In [ ]:
negatives_tokenized

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from gensim.corpora import Dictionary
import pyLDAvis
import pyLDAvis.gensim

In [ ]:
id2word = corpora.Dictionary(negatives_tokenized)
dictionary = id2word
dictionary = dictionary.filter_extremes(no_above=0.7)

texts = negatives_tokenized
corpus = [id2word.doc2bow(text) for text in texts]
ldamallet = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=2, id2word=id2word, iterations=100)

coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()

def compute_coherence_values(dictionary, corpus, texts, limit=8, start=2, step=1):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word, alpha=.91)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values


model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=4, limit=8, step=2)

max_y = max(coherence_values)  # Find the maximum y value
max_x = coherence_values.index(max(coherence_values))  # Find the x value corresponding to the maximum y value
#xmax = x[numpy.argmax(y)]

optimal_model = model_list[coherence_values.index(max(coherence_values))]

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word, R=10)
vis

In [ ]:
from nltk.corpus import webtext
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

finder = BigramCollocationFinder.from_words(negatives)
finder.nbest(BigramAssocMeasures.likelihood_ratio, 10)